#Imports

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import f1_score, make_scorer
from sklearn.preprocessing import MinMaxScaler
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install scikit-learn==1.2.2 ltcn

In [ ]:
pip install -U imbalanced-learn

#Model

In [ ]:
from ltcn.LTCN import LTCN
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.pipeline import Pipeline as imbpipeline

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis/encoded_A14I_dataset")

In [ ]:
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure
0,1,298.1,308.6,1551,42.8,0,0
1,0,298.2,308.7,1408,46.3,3,0
2,0,298.1,308.5,1498,49.4,5,0
3,0,298.2,308.6,1433,39.5,7,0
4,0,298.2,308.7,1408,40.0,9,0


#Define Functions

In [ ]:
def custom_f1(y_true, y_pred):
    return f1_score(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1), average="weighted")

In [ ]:
def custom_f1_minority(y_true, y_pred):
    classes_of_interest = [1, 2, 3, 4]
    return f1_score(y_true, np.argmax(y_pred, axis=1), labels=classes_of_interest, average="weighted")

In [ ]:
def custom_f1_majority(y_true, y_pred):
    classes_of_interest = [0]
    return f1_score(y_true, np.argmax(y_pred, axis=1), labels=classes_of_interest, average="weighted")

In [ ]:
def nested_cv_model(X, y, grid, classifier):

    outer_cv = StratifiedKFold(n_splits=5)

    train_errors_majority = []
    train_errors_minority = []

    test_errors_majority = []
    test_errors_minority = []

    train_wa = []
    test_wa= []
    cv_wa = []

    confusion = []
    aggregated_results = []

    all_cv_results = []

    for train_index, test_index in outer_cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        pipeline = imbpipeline(steps=[['smote', SMOTE()],
                                      ['scaler', MinMaxScaler()],
                                      ['classifier', classifier]])

        F1 = make_scorer(custom_f1)

        # inner CV for hyperparameter tuning
        inner_cv = StratifiedKFold(n_splits=5)

        #initialize grid
        grid_search = RandomizedSearchCV(pipeline,
                                    grid,
                                    scoring=F1,
                                    cv=inner_cv.split(X_train, y_train),
                                    n_jobs=-1)

        #one hot encode y_train here
        grid_search.fit(X_train, (np.eye(len(np.unique(y_train)))[np.unique(y_train, return_inverse=True)[1]]))

        Y_pred_train = grid_search.predict(X_train)
        train_errors_minority.append(custom_f1_minority(y_train, Y_pred_train))
        train_errors_majority.append(custom_f1_majority(y_train, Y_pred_train))

        Y_pred_test = grid_search.predict(X_test)
        test_errors_minority.append(custom_f1_minority(y_test, Y_pred_test))
        test_errors_majority.append(custom_f1_majority(y_test, Y_pred_test))

        train_wa.append(custom_f1(y_train, Y_pred_train))
        test_wa.append(custom_f1(y_test, Y_pred_test))
        cv_wa.append(grid_search.best_score_)

        #cv results for this fold
        fold_results = pd.DataFrame(grid_search.cv_results_)
        all_cv_results.append(fold_results)

        #confusion matrix for this fold
        cm = confusion_matrix(y_test, np.argmax(Y_pred_test, axis=1))
        confusion.append(cm)

        aggregated_results.append((grid_search.best_params_, grid_search.best_score_))

    all_cv_results = pd.concat(all_cv_results, ignore_index=True)

    # best parameters across all folds
    best_params_global = max(aggregated_results, key=lambda x: x[1])[0]
    print(best_params_global)

    best_classifier = classifier.set_params(**{k.replace('classifier__', ''): v for k, v in best_params_global.items()})

    #final pipeline with the best classifier
    final_pipeline = imbpipeline(steps=[['smote', SMOTE()],
                                        ['scaler', MinMaxScaler()],
                                        ['classifier', best_classifier]])

    # fit final pipeline on the entire dataset
    final_pipeline.fit(X, (np.eye(len(np.unique(y)))[np.unique(y, return_inverse=True)[1]]))

    # average scores across all outer folds
    average_train_score_majority = np.mean(train_errors_majority)
    average_test_score_majority = np.mean(test_errors_majority)

    average_train_score_minority = np.mean(train_errors_minority)
    average_test_score_minority = np.mean(test_errors_minority)

    average_train_wa = np.mean(train_wa)
    average_test_wa =  np.mean(test_wa)
    average_cv_wa = np.mean(cv_wa)

    return {'majority train':average_train_score_majority, 'majority test':average_test_score_majority}, {'minority train':average_train_score_minority,'minority test':average_test_score_minority}, {'weighted average train':average_train_wa, 'weighted average cv':average_cv_wa, 'weighted average test':average_test_wa}, confusion, final_pipeline, all_cv_results

#Execution

In [ ]:
feature_columns = ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
target_column = ['Machine failure']

X = df[feature_columns].to_numpy()
Y = df[target_column].to_numpy()

param_grid = {'classifier__phi': np.arange(0.5, 1, 0.1),
              'classifier__T': [5, 10, 15],
              'classifier__alpha': [0, 1.0E-2, 1.0E+2],
              'classifier__function':['tanh', 'sigmoid']}

ltcn = LTCN()

In [ ]:
majority_score, minority_score, weighted_average_score, confusion_matrix_ltcn, best_ltcn, cv_results_ltcn = nested_cv_model(X, Y, param_grid, ltcn)
print(majority_score, "\n", minority_score, "\n", weighted_average_score)

{'classifier__phi': 0.5, 'classifier__function': 'tanh', 'classifier__alpha': 0, 'classifier__T': 10}
{'majority train': 0.8903870076754185, 'majority test': 0.8704526086077082} 
 {'minority train': 0.36413531885596917, 'minority test': 0.3751317897140608} 
 {'weighted average train': 0.8741215794610497, 'weighted average cv': 0.8610924816044129, 'weighted average test': 0.8557230715999049}


In [ ]:
with open('/content/drive/MyDrive/Thesis/RLTCN_model.pkl', 'wb') as file:
    pickle.dump(best_ltcn, file)